In [1]:
from collections import defaultdict
import pandas as pd

## IDEA FOR CACHING: Check overlapping between the doc-ids of the query at the current turn and the doc-ids of queries of the previout turns

### Remapping the remapped

In [2]:
# Load qid and docid remapping dictionaries

# query id dict
qid_mapping_path = "/data3/muntean/DRhard/data/passage/dataset/queries.CASToriginal.QID2newID.test.tsv"
queries_df = pd.read_csv(qid_mapping_path, delimiter="\t", header=None)
print(len(queries_df))

# collection id dict
collection_mapping_path = "/data3/muntean/DRhard/data/passage/dataset/CASTcollectionPID2newID.tsv"
collection_df = pd.read_csv(collection_mapping_path, delimiter="\t", header=None)
print(len(collection_df))

479
38626614


In [3]:
# Create reverse dictionaries
queries_dict = dict(zip(queries_df[1], queries_df[0]))
collection_dict = dict(zip(collection_df[1], collection_df[0]))

In [4]:
results_star_dot_product_path = "/data3/muntean/DRhard/data/passage/official_runs/star/test.rank.tsv" # original queries
results_star_dot_product_df = pd.read_csv(results_star_dot_product_path, delimiter="\t", header=None)
print(len(results_star_dot_product_df))
results_star_dot_product_df.head()

47900


,0,1,2
0,0,3878347,1
1,0,291003,2
2,0,789620,3
3,0,2954451,4
4,0,5625372,5


In [5]:
with open ('/data3/muntean/conversational-cache/data/star-ranking/CAST-original-queries-star-dotp-ranking-top100.tsv', "w") as f:
    for index, row in results_star_dot_product_df.iterrows():
        f.write("{}\t{}\t{}\n".format(queries_dict[row[0]], collection_dict[row[1]], row[2]))

# Starts here

In [6]:
#1. loading data: load the first k (e.g. k=10) docs for each utterance in a dictionary

map_utt_docs = defaultdict(list) #dictionary: key = utt_id, value = list of top-k results

max_k = 100

#server
path = "../data/star-ranking/"
with open(path+"CAST-original-queries-star-dotp-ranking-top100.tsv", 'r') as f:
    next(f)
    for line in f:
        line_fields = line.rstrip().split("\t")
        utt_id = line_fields[0]
        doc_id = line_fields[1]
        rank = int(line_fields[2])
        if rank <= max_k: 
            map_utt_docs[utt_id].append(doc_id)

In [7]:
# map_utt_docs

In [8]:
### REMMBER to check the YEAR

qid_rel_dict = defaultdict(int)
path = "../data/CAST_qrels/"
with open(path+"qrels-docs.2019.txt", 'r') as f:
    for line in f:
        line_fields = line.rstrip().split(" ")
        utt_id = line_fields[0]
        doc_id = line_fields[2]
        relevance = int(line_fields[3])
        if relevance > 0:
            qid_rel_dict[utt_id]+=1

In [9]:
#functions to check the intersection between top-k results of current turn and top-k results of the previous turns 

def intersection(lst1, lst2):
    return set(lst1).intersection(lst2)

#@deprecated
def compute_overapping_documents(conv_id, curr_turn, f):
    total_overlapping = 0
    
    docs_current_turn =  map_utt_docs[str(conv_id)+"_"+str(curr_turn)]
    #print("current turn: "+str(curr_turn))
    #print(docs_current_turn)

    docs_so_far = {}
    for turn_id in range(1, curr_turn):
        if str(conv_id)+"_"+str(turn_id) in map_utt_docs:
            docs_so_far = map_utt_docs[str(conv_id)+"_"+str(turn_id)]
            #print("so_far: "+str(turn_id))
            #print(docs_so_far)
            #print(intersection(docs_so_far, docs_current_turn))
            #print(len(intersection(docs_so_far, docs_current_turn)))
            total_overlapping += len(intersection(docs_so_far, docs_current_turn))
        else:
            f.write(str(conv_id)+"_"+str(turn_id)+" not found\n")

    f.write(str(conv_id)+"_"+str(curr_turn)+"\t"+str(total_overlapping)+"\n")


def compute_overapping_documents_set(conv_id, curr_turn, f, qid_rel_dict):
    
    total_overlapping = 0
    
    utt_id = str(conv_id)+"_"+str(curr_turn)
    docs_current_turn =  map_utt_docs[utt_id]

    docs_so_far = set()
    for turn_id in range(1, curr_turn):
        if str(conv_id)+"_"+str(turn_id) in map_utt_docs:
            my_list = map_utt_docs[str(conv_id)+"_"+str(turn_id)]
            for el in my_list:                
                docs_so_far.add(el)
        else:
            f.write(str(conv_id)+"_"+str(turn_id) + " not found\n")
    total_overlapping = len(intersection(docs_so_far, docs_current_turn))
    # print(docs_so_far, docs_current_turn, total_overlapping)
    normalized_overlapping = "not in qrel"
    if utt_id in qid_rel_dict:
        if qid_rel_dict[utt_id]>0:
            normalized_overlapping = len(intersection(docs_so_far, docs_current_turn))/qid_rel_dict[utt_id]
        else:
            normalized_overlapping = "0 relevants"
    
    if normalized_overlapping != "not in qrel":
        f.write(utt_id+"\t"+str(total_overlapping)+"\t"+str(qid_rel_dict[utt_id])+"\t"+str(normalized_overlapping)+"\n")
    

In [10]:
# main (with sets, no duplicates): to run after loading data

path = "../data/proof_of_concepts_preliminary_results/retrieved/star-top100/"
with open(path+"normalized-overalapping-sets-docs.CAST_original_star.txt", 'w') as f:
    for el in map_utt_docs:
        conv_id = int(el.split("_")[0])
        curr_turn = int(el.split("_")[1])
        compute_overapping_documents_set(conv_id, curr_turn, f, qid_rel_dict)